In [4]:
"""
Transfer All NSIDC Ground Truth with MODIS Matched Data to MySQL
With year/day/day_label extended for indexing
Author: Qi Liu
Date: Oct.09th, 2016
"""

import pandas as pd
from pandas.tools.plotting import scatter_matrix
import numpy as np
import glob
import os
from pandas.io import sql
import MySQLdb
import time

start_time = time.time()

dpath = '/home/mirabot/googledrive/deeplearning/seaiceprj/modis/test/'#'/home/mirabot/googledrive/IceData_DL/initialtest/data/'
rpath = '/home/mirabot/googledrive/deeplearning/seaiceprj/modis/results/'#'/home/mirabot/googledrive/IceData_DL/initialtest/data/'

mode = 3

flist = glob.glob(dpath+'*.csv')
rlist = glob.glob(rpath+'*.csv')

print rlist
#mysql connection, install sudo apt-get install python2.7-mysqldb
con = MySQLdb.connect(host="localhost",port=3306,user="mira67",passwd="1234",db="nsidcgt")
if mode == 0:#record NSIDC ground truth data
    print "Write NSIDC GT"
    for tf in flist:
        df = pd.read_csv(tf, sep=",", skiprows=1, names = ["FID","Id","b1","b2","b3","b4","b5","b6","b7","b8","Rowid","MP_F","I_F","W_F",])
        nsplit = os.path.basename(tf).split('_')
        df_len = len(df['FID'])
        site_name = np.chararray((df_len), itemsize=6)
        site_name[:] = nsplit[0]
        year = np.zeros(df_len)
        year[:] = int(nsplit[1][1:5])
        day = np.zeros(df_len)
        day[:] = int(nsplit[1][5:8])
        day_label = np.chararray(df_len, itemsize=1)
        if len(nsplit[1]) == 9:
            day_label[:] = nsplit[1][8]
        else:
            day_label[:] = 'N'
        df['site'] = pd.Series(site_name, index=df.index)
        df['year'] = pd.Series(year, index=df.index)
        df['day'] = pd.Series(day, index=df.index)
        df['day_label'] = pd.Series(day_label, index=df.index)
        #record data to database
        df.to_sql(con=con, name=site_name[0], if_exists='append', flavor='mysql')

elif mode == 1:#record MODIS data
    print "Write MOD09"
    for tf in flist:
        print tf
        df = pd.read_csv(tf, sep=",", skiprows=1, names = ["year","month","day","nrow","ncol","qc","cloud","b1","b2","b3","b4","b5","b6","b7"])
        df.to_sql(con=con, name='mod500', if_exists='append', flavor='mysql')

elif mode == 2:#not use pandas
    sql = """LOAD DATA LOCAL INFILE '{}' 
    INTO TABLE mod500 
    FIELDS TERMINATED BY ','
    OPTIONALLY ENCLOSED BY '"'
    LINES TERMINATED BY '\\n'
    IGNORE 1 LINES;"""
    
    for tf in flist:
        print tf
        try:
             cursor = con.cursor()
             cursor.execute(sql.format(tf))
             con.commit()
        except Exception,e:
             print str(e)
             # Rollback in case there is any error
             con.rollback()
        print 'complete file %s' % (tf)
        
elif mode == 3:#write results to database
    sql = """LOAD DATA LOCAL INFILE '{}' 
    INTO TABLE mod500res_nocloud_b_m3 
    FIELDS TERMINATED BY ','
    OPTIONALLY ENCLOSED BY '"'
    LINES TERMINATED BY '\\n'
    IGNORE 1 LINES;"""
    
    for tf in rlist:
        print tf
        try:
             cursor = con.cursor()
             cursor.execute(sql.format(tf))
             con.commit()
        except Exception,e:
             print str(e)
             # Rollback in case there is any error
             con.rollback()
        print 'complete file %s' % (tf)

# disconnect from server
con.close()
os.system('espeak "Congratulations, done with database"')
print("---total time: %s seconds ---" % (time.time() - start_time))

['/home/mirabot/googledrive/deeplearning/seaiceprj/modis/results/mod500results_m3.csv']
/home/mirabot/googledrive/deeplearning/seaiceprj/modis/results/mod500results_m3.csv
complete file /home/mirabot/googledrive/deeplearning/seaiceprj/modis/results/mod500results_m3.csv
---total time: 38.6068298817 seconds ---
